# Wie kann ein Landwirt entscheiden?

## Die Story

<div class="alert alert-block alert-success">

Ein Landwirt möchte entscheiden, ob ein man heute mit der Getreideernte beginnen möchte.
 
Das hängt z.B. von der Regenwahrscheinlichkeit und dem Feuchtegehalt des Getreides ab.
   
Seine Entscheidungen in der Vergangenheit hat der Landwirt in einer Tabelle aufgezeichnet.


<div class="alert alert-block alert-info">

Zur Vorbereitung werden wieder einige notwendige Bibliotheken sowie die Daten eingelesen.
    
**Führe also die folgenden beiden Zellen aus:**

In [ ]:
import pandas as pd
import plotly.express as px
from PyTree import ClassificationTree as ct
import ipywidgets as widgets

color_map = {'ernten': 'green', 'warten': 'red'}

data_x = "Feuchtegrad"
data_y = "Regen"
target = "Aktion"
breite=800
hoehe=600

In [ ]:
df_ernten = pd.read_csv("Daten/Ernteentscheidung.csv", sep=";")
display(df_ernten)

In [ ]:
max_x = max(df_ernten ["Feuchtegrad"])
min_x = min(df_ernten ["Feuchtegrad"])
max_y = max(df_ernten ["Regen"])
min_y = min(df_ernten ["Regen"])

delta_x = (max_x - min_x)
delta_y = (max_y - min_y)

bereich_x = [int(min_x - delta_x*0.05), int(max_x + delta_x*0.15)]
bereich_y = [int(min_y - delta_y*0.05), int(max_y + delta_y*0.05)]

## Entscheidungsbaum

<div class="alert alert-block alert-success">

Betrachten wir das Streudiagramm mit dem Schwellenwert `Regen 45`, in dem die Anzahl Fehlklassifikationen eingetragen ist: 

In [ ]:
schwellenwert = 45

def fehlersumme_l(threshold):
    tree = ct.DecisionTree(target=target, data=df_ernten)
    tree.manual_split(attribute="Regen", threshold=threshold, node_nr=1)
    return tree.calculate_errors(data=df_ernten)


fig = px.scatter(
    df_ernten,
    x=data_x,
    y=data_y,
    color=target,
    color_discrete_map=color_map,
    width=breite,
    height=hoehe,
    range_x=bereich_x,
    range_y=bereich_y,
)

fig.update_traces(marker=dict(size=10, line=dict(width=1, color="black")))

fig.update_layout(
    shapes=[
        dict(
            type="line",
            yref="y1",
            y0=schwellenwert,
            y1=schwellenwert,
            xref="x1",
            x0=bereich_x[0],
            x1=bereich_x[1],
            line=dict(color="Red", width=1),
        )
    ]
)

fig.add_annotation(
    x=16,
    y=schwellenwert,
    text="Schwellenwert Regen = " + str(schwellenwert),
    showarrow=False,
    xshift=15,
)

fig.add_annotation(
    x=16,
    y=schwellenwert,
    text="Fehler: " + str(fehlersumme_l(schwellenwert)),
    showarrow=False,
    yshift=15,
)


fig.show()

<div class="alert alert-block alert-warning">

#### *Aufgabe:* 
    
Beschreibe, wie der Wert `39` als Fehleranzahl entsteht!
</div>

<div class="alert alert-block alert-success">
    
Wenn du dir den zugehörigen Entscheidungsbaum zeigen lässt, kannst du verschiedene Informationen ablesen:

In [ ]:
tree = ct.DecisionTree(target=target, data=df_ernten)
tree.manual_split(attribute="Regen", threshold=45, node_nr=1)
tree.print_tree()

<div class="alert alert-block alert-warning">

#### *Aufgabe:* 
    
Schreibe jetzt hier Deine Beobachtung hin.
    
- In dem blauen Rechteck:
    - ???
- In dem weissen Rechteck Nr. 1:
   - ??? 
- In dem linken gelben Rechteck Nr. 2:
   - ??? 
- In dem rechten gelben Rechteck Nr. 3:
    - ???
</div>

<div class="alert alert-block alert-success">

Du kannst dir hier für verschiedene Schwellenwerte den jeweiligen Entscheidungsbaum ansehen:

In [ ]:
start = (min_y + max_y) // 2

slider_l = widgets.BoundedFloatText(
    value=start,
    min=int(min_y+0.5),
    max=int(max_y-0.5),
    step=0.5,
    description="",
    layout=widgets.Layout(width="30%"),
)

def makeTree_l(schwellenwert):
    tree = ct.DecisionTree(target=target, data=df_ernten)
    tree.manual_split(attribute="Regen", threshold=schwellenwert, node_nr=1)
    return tree

def on_value_change_l(event):

    with output_l:
        output_l.clear_output()
        sw_l = event["new"]
        tree = makeTree_l(sw_l)
        display (tree.print_tree())


slider_l.observe(on_value_change_l, names="value")

output_l = widgets.Output ();
display(
    widgets.HBox([widgets.Label("Schwellenwert für Regen"), slider_l, output_l])
)
sw_l = start
tree = makeTree_l(sw_l)
with output_l:
    display (tree.print_tree())

<div class="alert alert-block alert-warning">

#### *Aufgabe:* 
    
Suche jetzt für die Variable `Regen` den besten Schwellenwert!

1. Für einen Schwellenwert von 30 werden 9 Insekten (1 Marienkäfer, 8 Raupen) falsch klassifiziert. 
2. Für einen Schwellenwert von 70 werden 6 Insekten (alles Marienkäfer) falsch als Raupe klassifiziert. 

<div class="alert alert-block alert-success">

Wir können jetzt auch den Feuchtegrad als Kriterium nutzen:

In [ ]:
start = (min_x + max_x) // 2

slider_r = widgets.BoundedFloatText(
    value=start,
    min=int(min_x+0.5),
    max=int(max_x-0.5),
    step=0.5,
    description="",
    layout=widgets.Layout(width="30%"),
)

def makeTree_r(schwellenwert):
    tree = ct.DecisionTree(target=target, data=df_ernten)
    tree.manual_split(attribute="Feuchtegrad", threshold=schwellenwert, node_nr=1)
    return tree

def on_value_change_r(event):

    with output_r:
        output_r.clear_output()
        sw_r = event["new"]
        tree = makeTree_r(sw_r)
        display (tree.print_tree())


slider_r.observe(on_value_change_r, names="value")

output_r = widgets.Output()

display(
    widgets.HBox([widgets.Label("Schwellenwert für Feuchtegrad"), slider_r, output_r])
)
sw_r = start
tree = makeTree_r(sw_r)
with output_r:
    display (tree.print_tree())

<div class="alert alert-block alert-warning">

#### *Aufgabe:* 
    
Suche jetzt für die Variable `Feuchtegrad` den besten Schwellenwert!

Für einen Schwellenwert von 14.5 entstehen 7 Fehler. Das ist der beste Schwellenwert.

<div class="alert alert-block alert-success">

Hier kannst du dir nochmal verschiedene Entscheidungsbäume ansehen. Du kannst
    
- wählen, welche Variable du als Prädiktorvariable nutzen möchtest.
- den Schwellenwert einstellen.

In [ ]:
def show_tree(attribute, schwellenwert):

    tree = ct.DecisionTree(target=target, data=df_ernten)
    tree.manual_split(attribute=attribute, threshold=schwellenwert, node_nr=1)
    tree.print_tree()
    display(tree.tree_graph)

    return tree


tree_widget = widgets.interactive(
    show_tree,
    {"manual": True, "manual_name": "Erstelle Tree"},
    attribute = ["Regen","Feuchtegrad"],
    schwellenwert = (0, 100, 0.5)
)
tree_widget

## Weitere Datensplits

<div class="alert alert-block alert-success">
    
Wir haben jetzt die Daten optimal anhand des Feuchtegrades klassifiziert.
    
Denn der optimale Datensplit (Feuchtegrad = 14.5) erzeugte nur 7 Fehler. Diese Fehler erkennt man in dem Knoten 3:
    
- Alle Daten werden als *warten* interpretiert, obwohl 7 Daten als *ernten* gelabled waren.
   
Die Idee ist jetzt, den Knoten mit der Nummer 3 weiter aufzuteilen:
    
Also:

- Für die Daten mit `Feuchtegehalt > 14.` suchen wir nach einem Schwellenwert für die Regenwahrscheinlichkeit, um die insgesamt 49 Daten zu unterscheiden. Dieser Schwellenwert soll also nur für die daten im Knoten Nr. 3 zuständig sein.

<div class="alert alert-block alert-warning">

#### *Aufgabe:* 
    
Ändere in dem folgenden Programmabschnitt den Wert für `schwellenwertFuerRegen` geeignet ab, also so, dass möglichst wenige Fehlklassifikationen entstehen.
</div>

In [ ]:
def show_tree(attribute, schwellenwert):

    tree = ct.DecisionTree(target=target, data=df_ernten)
    tree.manual_split(attribute='Feuchtegrad', threshold=14.5, node_nr=1)
    tree.manual_split(attribute='Regen', threshold=schwellenwert, node_nr=3)

    tree.print_tree()
    
    print("Fehler:", tree.calculate_errors(data=df_ernten))

    display(tree.tree_graph)

    return tree

tree_widget = widgets.interactive(
    show_tree,
    {"manual": True, "manual_name": "Erstelle Tree"},
    attribute = ["Regen"],
    schwellenwert = (10, 90, 0.5)
)
tree_widget

## Der optimale Entscheidungsbaum mit zwei Stufen wird automatisch erstellt.

<div class="alert alert-block alert-success">

Bisher haben wir den besten Entscheidungsbaum selber erstellt, indem wir in mehreren Stufen die jeweils besten Schwellenwerte genutzt haben, um weitere Datensplits zu erzeugen.
    
Es gibt eine Bibliothek, die diesen Vorgang automatisiert. Sie funktioniert nach genau diesem Prinzip:

In [ ]:
tree_all = ct.DecisionTree(target = 'Aktion', data = df_ernten)

#Entscheidungsbaum erstellen 
tree_all.grow_tree(df_ernten, 'Aktion', max_depth = 2)

#Entscheidungsbaum ausgeben
tree_all.print_tree()

<div class="alert alert-block alert-success">

Das zugehörige Streudiagramm:

In [ ]:
import plotly.express as px

vertikal = 14.6
horizontal = 79.0

tree = ct.DecisionTree(target="Aktion", data=df_ernten)

#fehlersumme = calc_errors(hor, tree, "Länge")
#fehlersumme = tree.calculate_errors(data = df_kaefer) 

color_map = {'ernten': 'green', 'warten': 'red'}

fig = px.scatter(
    df_ernten,
#    df_testdaten,
    x="Feuchtegrad",
    y="Regen",
    color="Aktion",
    color_discrete_map=color_map,
    width=800,
    height=600,
    range_x=bereich_x,
    range_y=bereich_y,
    #size = 0.01
)

fig.update_traces(marker=dict(  #size=5, 
    line=dict(width=1, color='black')))


fig.update_layout(shapes=[
    dict(
        type="line",
        yref="y1",
        x0=vertikal,
        x1=vertikal,
        xref="x1",
        y0=bereich_y[0],
        y1=horizontal,
        line=dict(color="Red", width=1),
    ),
    dict(
        type="line",
        yref="y1",
        x0=bereich_x[0],
        x1=bereich_x[1],
        xref="x1",
        y0=horizontal,
        y1=horizontal,
        line=dict(color="Red", width=1),
    ),
    dict(
        type="rect",
        yref="y1",
        x0=bereich_x[0],
        x1=vertikal,
        xref="x1",
        y0=bereich_y[0],
        y1=bereich_y[1],
        fillcolor=color_map["ernten"],
        opacity=0.3,
        layer="below",
        line_width=0,
    ),
    dict(
        type="rect",
        yref="y1",
        x0=vertikal,
        x1=bereich_x[1],
        xref="x1",
        y0=bereich_y[0],
        y1=horizontal,
        fillcolor=color_map["warten"],
        opacity=0.3,
        layer="below",
        line_width=0,
    ),
    dict(
        type="rect",
        yref="y1",
        x0=vertikal,
        x1=bereich_x[1],
        xref="x1",
        y0=horizontal,
        y1=bereich_y[1],
        fillcolor=color_map["ernten"],
        opacity=0.3,
        layer="below",
        line_width=0,
    ),
])

fig.show()